In [8]:
import numpy as np
import pandas as pd
import os
from collections import defaultdict
from tqdm import tqdm
from pprint import pprint

from Bio import Entrez
Entrez.email = "sample@bioinf.me"

from functools import lru_cache

pd.options.mode.chained_assignment = None  # default='warn'
tqdm.pandas()

## For meta data

In [9]:
@lru_cache(maxsize=4096)
def return_gene_by_id(uid):
    handle = Entrez.esummary(db="gene", id=uid)
    uid_record = Entrez.read(handle)
    handle.close()
    uid_summary = uid_record["DocumentSummarySet"]["DocumentSummary"][0]
    return uid_summary["Name"]


@lru_cache(maxsize=4096)
def return_gene_by_rsid(snp_id):
    answer = []
    record = Entrez.read(
        Entrez.elink(dbfrom="snp", id=snp_id.replace("rs", ""), db="gene")
    )
    results = record[0]["LinkSetDb"]
    if len(results) < 1:
        return None
    results = results[0]["Link"]
    for result in results:
        uid = result["Id"]
        gene_name = return_gene_by_id(uid)
        answer.append(gene_name)
    return ",".join(answer)

In [13]:
cutoff = 0.05
top_features = ["I9_HYPTENSPREG", "GEST_DIABETES", "O15_PRETERM"]

datas = []
all_datas = defaultdict()
ns = defaultdict()

for feat in top_features:

    cur_dir = "analysis_n_" + feat

    cur_files = [f for f in os.listdir(f"{cur_dir}/data")]
    metal_data_name = [f for f in cur_files if f[-4:] == ".TBL" and "extended_" in f][0]
    mtag_data_name = "_mtag_meta.txt"
    ukb_data_name = [f for f in cur_files if "gwas.imputed_v3" in f][0]
    finn_data_name = [f for f in cur_files if "hg19lifted" in f][0]

    metal_data = pd.read_csv(f"{cur_dir}/data/{metal_data_name}", sep="\t").add_prefix(
        "metal_"
    )
    mtag_data = (
        pd.read_csv(f"{cur_dir}/data/{mtag_data_name}", sep="\t")
        .rename(
            columns={
                "SNP": "rsid",
                "CHR": "chr",
                "BP": "pos",
                "A1": "ref",
                "A2": "alt",
                "meta_freq": "maf",
                "mtag_beta": "beta",
                "mtag_se": "se",
                "mtag_z": "z",
                "mtag_pval": "pval",
            }
        )
        .add_prefix("mtag_")
    )
    ukb_data = pd.read_csv(f"{cur_dir}/data/{ukb_data_name}", sep="\t").add_prefix(
        "ukb_"
    )
    finn_data = pd.read_csv(f"{cur_dir}/data/{finn_data_name}", sep="\t").add_prefix(
        "finn_"
    )

    metal_data["metal_significant"] = (
        metal_data.metal_pval < cutoff / metal_data.shape[0]
    )
    mtag_data["mtag_significant"] = mtag_data.mtag_pval < cutoff / mtag_data.shape[0]
    ns[feat] = [
        metal_data.shape[0],
        mtag_data.shape[0],
        ukb_data.shape[0],
        finn_data.shape[0],
    ]

    data = pd.merge(
        metal_data, mtag_data, how="outer", left_on="metal_rsid", right_on="mtag_rsid"
    )
    all_datas[feat] = data

    data = data[
        data.mtag_significant
        | data.metal_significant
        | (data.mtag_significant.isna() & data.metal_significant)
    ]
    data = pd.merge(
        data, ukb_data, how="left", left_on="metal_rsid", right_on="ukb_rsid"
    )
    data = pd.merge(
        data, finn_data, how="left", left_on="metal_rsid", right_on="finn_rsid"
    )
    data = data.rename(
        columns={
            "metal_rsid": "rsid",
            "metal_chr": "chr",
            "metal_pos": "pos",
            "metal_ref": "ref",
            "metal_alt": "alt",
        }
    )
    data["trait"] = feat
    data["gene"] = data.rsid.progress_apply(return_gene_by_rsid)
    data = data[
        [
            "rsid",
            "chr",
            "gene",
            "pos",
            "ref",
            "alt",
            "metal_pval",
            "mtag_pval",
            "ukb_pval",
            "finn_pval",
            "trait",
            "metal_significant",
            "mtag_significant",
        ]
    ]
    datas.append(data)

/home/achangalidi/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/achangalidi/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/achangalidi/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  1.89it/s]


## Prepare to sort

In [17]:
import re

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]
kk = list(all_datas[feat].metal_chr.astype(str).unique())
kk.sort(key=natural_keys)
order = {i:j for i, j in zip(kk, range(len(kk)))}
order

{'1': 0,
 '2': 1,
 '3': 2,
 '4': 3,
 '5': 4,
 '6': 5,
 '7': 6,
 '8': 7,
 '9': 8,
 '10': 9,
 '11': 10,
 '12': 11,
 '13': 12,
 '14': 13,
 '15': 14,
 '16': 15,
 '17': 16,
 '18': 17,
 '19': 18,
 '20': 19,
 '21': 20,
 '22': 21,
 'X': 22}

## Sort and polish

In [18]:
STARTING_GROUP = 0

def make_group_col(df, col, group_col='group'):
    global STARTING_GROUP
    df[group_col] = None
    df.groupby((df[col].shift() != df[col]).cumsum())
    for k, v in df.groupby((df[col].shift() != df[col]).cumsum()):
        df.loc[v.index,group_col] = STARTING_GROUP+k
    STARTING_GROUP += k

In [19]:
datas[0]

,rsid,chr,gene,pos,ref,alt,metal_pval,mtag_pval,ukb_pval,finn_pval,trait,metal_significant,mtag_significant
0,rs10882398,10,PLCE1,95892788,T,A,8.943000e-09,NaN,0.005538,3.480000e-09,I9_HYPTENSPREG,True,NaN
1,rs10882397,10,PLCE1,95892659,C,A,1.456000e-08,7.309473e-09,0.003695,1.950000e-08,I9_HYPTENSPREG,False,True
2,rs16998073,4,None,81184341,A,T,9.128000e-09,NaN,0.035201,1.910000e-11,I9_HYPTENSPREG,True,NaN
3,rs932764,10,PLCE1,95895940,A,G,1.388000e-08,7.024045e-09,0.002932,2.980000e-08,I9_HYPTENSPREG,False,True
4,rs12509595,4,None,81182554,T,C,1.575000e-08,7.070137e-09,0.043569,2.570000e-11,I9_HYPTENSPREG,False,True
5,rs35954793,4,FGF5,81188513,C,A,6.145000e-09,2.680313e-09,0.029762,1.600000e-11,I9_HYPTENSPREG,True,True
6,rs167479,19,RGL3,11526765,G,T,5.233000e-09,2.223811e-09,0.038050,5.130000e-12,I9_HYPTENSPREG,True,True


In [20]:
for i in range(len(datas)):
    datas[i].pos = datas[i].pos.astype(int)
    datas[i].chr = datas[i].chr.astype(str)
    datas[i]['chr_order'] = datas[i].chr.apply(lambda x: order[x])
    datas[i]['chr_gene'] = datas[i].chr + '_' + datas[i].gene.astype(str)
    datas[i] = datas[i].sort_values(by=['chr_order', 'pos'])
    make_group_col(datas[i], 'chr_gene', 'group')
    datas[i] = datas[i].drop(['chr_order', 'chr_gene'], axis = 1)

In [21]:
best_datas = []
PVAL_COL = 'metal_pval'
for data in datas:
    data
    best = data.groupby('group').agg(
        pval=pd.NamedAgg(column=PVAL_COL, aggfunc="min"))
    filtered_data = data[data[PVAL_COL].isin(best.pval)]
    # может произойти что в одной хромосоме данный пвал макс, а в другой нет 
    # - важно взять нужную и только ее! 
    flags = []
    for key, value in filtered_data.iterrows():
        flags.append(best[best.pval==value[PVAL_COL]].index[0]==value.group)
    best_datas.append(filtered_data[flags])
final = pd.concat(best_datas)
final.to_csv('./data/meta_top.tsv', sep='\t', index=False)
print(final.shape)
final

(9, 14)


,rsid,chr,gene,pos,ref,alt,metal_pval,mtag_pval,ukb_pval,finn_pval,trait,metal_significant,mtag_significant,group
2,rs16998073,4,None,81184341,A,T,9.128000e-09,NaN,0.035201,1.910000e-11,I9_HYPTENSPREG,True,NaN,1
5,rs35954793,4,FGF5,81188513,C,A,6.145000e-09,2.680313e-09,0.029762,1.600000e-11,I9_HYPTENSPREG,True,True,2
0,rs10882398,10,PLCE1,95892788,T,A,8.943000e-09,NaN,0.005538,3.480000e-09,I9_HYPTENSPREG,True,NaN,3
6,rs167479,19,RGL3,11526765,G,T,5.233000e-09,2.223811e-09,0.038050,5.130000e-12,I9_HYPTENSPREG,True,True,4
33,rs36090025,10,TCF7L2,114774433,A,C,3.483000e-15,2.715405e-15,0.002077,1.910000e-18,GEST_DIABETES,True,True,5
101,rs7945617,11,None,92700287,T,C,2.798000e-31,2.037698e-31,0.036522,1.110000e-56,GEST_DIABETES,True,True,6
2,rs10830963,11,MTNR1B,92708710,C,G,4.530000e-41,NaN,0.136495,7.860000e-84,GEST_DIABETES,True,NaN,7
4,rs138109677,11,None,92725992,A,AATGTT,7.213000e-15,NaN,0.440159,7.060000e-30,GEST_DIABETES,True,NaN,8
0,rs2963457,5,None,157907974,C,T,6.520000e-09,4.715197e-09,0.000204,1.940000e-06,O15_PRETERM,True,True,9


Next, you should remove unnecessary SNPs

In [30]:
_final = pd.read_csv('./data/meta_top_short.csv', sep='\t')
_final

,rsid,chr,gene,pos,ref,alt,metal_pval,mtag_pval,ukb_pval,finn_pval,trait,metal_significant,mtag_significant,effect
0,rs35954793,4,FGF5,81188513,C,A,6.145000e-09,2.680313e-09,0.029762,1.600000e-11,I9_HYPTENSPREG,True,True,FGF5 : Intron Variant
1,rs10882398,10,PLCE1,95892788,T,A,8.943000e-09,NaN,0.005538,3.480000e-09,I9_HYPTENSPREG,True,NaN,PLCE1 : Intron Variant
2,rs167479,19,RGL3,11526765,G,T,5.233000e-09,2.223811e-09,0.038050,5.130000e-12,I9_HYPTENSPREG,True,True,RGL3 : Missense Variant
3,rs36090025,10,TCF7L2,114774433,A,C,3.483000e-15,2.715405e-15,0.002077,1.910000e-18,GEST_DIABETES,True,True,TCF7L2 : Intron Variant
4,rs10830963,11,MTNR1B,92708710,C,G,4.530000e-41,NaN,0.136495,7.860000e-84,GEST_DIABETES,True,NaN,MTNR1B : Intron Variant
5,rs2963457,5,EBF1,157907974,C,T,6.520000e-09,4.715197e-09,0.000204,1.940000e-06,O15_PRETERM,True,True,NaN


In [31]:
traits = _final.trait.unique()
s=''
for t in traits:
    s+="c('"+"', '".join(list(_final[_final.trait==t].rsid))+"'),\n"
print(s[:-2], end='\n\n')
s=''
for t in traits:
    s+="c('"+"', '".join(list(_final[_final.trait==t].gene))+"'),\n"
print(s[:-2], end='\n\n')
s=''
# for t in traits:
#     s+=f"'{DIR}/{next(filter(lambda f: t in f and 'flc_fmaf1' in f, files))}',\n"
# print(s[:-2], end='\n\n')
# s=''
for t in traits:
    s+=f"'_MET_{t}',\n"
print(s[:-2], end='\n\n')

c('rs35954793', 'rs10882398', 'rs167479'),
c('rs36090025', 'rs10830963'),
c('rs2963457')

c('FGF5', 'PLCE1', 'RGL3'),
c('TCF7L2', 'MTNR1B'),
c('EBF1')

'_MET_I9_HYPTENSPREG',
'_MET_GEST_DIABETES',
'_MET_O15_PRETERM'



## Sort in chr:pos ascending order

In [29]:
for t in top_features:
    d = all_datas[t].rename(columns={'metal_rsid': 'rsid', 'metal_chr': 'chr', 'metal_pos': 'pos', 'metal_ref': 'ref', 'metal_alt': 'alt', 'metal_pval':'pval'})
    d.pos = d.pos.astype(int)
    d.chr = d.chr.astype(str)
    d['chr_order'] = d.chr.apply(lambda x: order[x])
    kkkk = d.sort_values(by=['chr_order', 'pos'])[['rsid', 'chr', 'pos', 'ref', 'alt', 'pval']]
    cur_f = f"meta_special/{t}.tsv"
    kkkk.to_csv(cur_f, sep='\t', index=False)